In [ ]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader= TextLoader("speech.txt")
text_documents= loader.load()
text_documents

In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()
load_dotenv("../chatbot/.env")

os.environ["OLLAMA_API_KEY"]= os.getenv("OLLAMA_API_KEY")
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY") 

In [ ]:
# Web Based Loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()
text_documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("attention.pdf")
docs= loader.load()
docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

documents= text_splitter.split_documents(docs)
documents[:5]

In [ ]:
## Vector Embedding and Store
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings= OllamaEmbeddings(model="mxbai-embed-large")
db= Chroma.from_documents(
    documents= documents,
    embedding= embeddings,
)
db

In [ ]:
## Vector Database
query= "What is attention is all you need"

result= db.similarity_search(query)
result

In [ ]:
result[0].page_content

In [ ]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents[:30], OllamaEmbeddings(model="mxbai-embed-large"))

In [ ]:
## Vector Database
query= "What is attention is all you need"

result= db.similarity_search(query)
result[0].page_content

In [ ]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}"""
)

In [ ]:
## Chain Instruction
from langchain_community.chat_models import ChatOllama
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain= create_stuff_documents_chain(
    llm= ChatOllama(model="llama2"),
    prompt= prompt,
)